In [2]:
import edhec_risk_kit as erk
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthBegin
import plotly.graph_objs as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


START LAB 201

In [2]:
brka_d = pd.read_csv('data/brka_d_ret.csv', parse_dates=True, index_col=0)
brka_m = brka_d.resample('1M').apply(erk.cumulate)
brka_m.index -= MonthBegin(1)
fff = erk.get_df(filename='data/F-F_Research_Data_Factors_m.csv', start_period=None, end_period=None,
                 format='%Y%m', reqd_strategies=None)
rr_capm = erk.regress(brka_m, fff[['Mkt-RF', 'RF']], '1990', rfcol='RF')
rr_ffm = erk.regress(brka_m, fff, '1990', rfcol='RF')
rr_ffm_nint = erk.regress(brka_m, fff, '1990', rfcol='RF', intercept=False)
print(rr_ffm.summary())
print(rr_ffm_nint.summary())

                            OLS Regression Results                            
Dep. Variable:                   BRKA   R-squared:                       0.317
Model:                            OLS   Adj. R-squared:                  0.311
Method:                 Least Squares   F-statistic:                     53.29
Date:                Mon, 18 May 2020   Prob (F-statistic):           2.59e-28
Time:                        09:59:09   Log-Likelihood:                 567.01
No. Observations:                 348   AIC:                            -1126.
Df Residuals:                     344   BIC:                            -1111.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.7096      0.063     11.350      0.0

START LAB 202

In [3]:
ind = erk.get_df('data/ind30_m_vw_rets.csv', start_period='2000', format='%Y%m', mode='return', end_period=None, reqd_strategies=None)
mgr_r = 0.3*ind["Beer"] + .5*ind["Smoke"] + 0.2*np.random.normal(scale=0.15/(12**.5), size=ind.shape[0])
weights = erk.style_analyze(mgr_r, ind, start_period='2000')
style_coeff = erk.regress(mgr_r, ind, intercept=False, excess_mkt=False).params


PLOT FUNCTION

In [4]:
def plot_style(wts_coeff: pd.Series):
    wts_coeff.sort_values(ascending=False, inplace=True)
    wt_data = [go.Bar(x=wts_coeff.index, y=wts_coeff)]
    pyo.iplot(wt_data)

STYLE MATCHING PLOT USING QUADRATIC OPTIMIZER - MORE ACCURATE

In [5]:
plot_style(weights)

STYLE MATCHING PLOT USING REGRESSION COEFF

In [6]:
plot_style(style_coeff)

STYLE ANALYZE OF BRKA_M FOR LAST DECADE FROM 2009

In [7]:
wts_09 = erk.style_analyze(brka_m, ind, start_period='2009')
plot_style(wts_09)



PENDING WORK - DOWNLOAD ANY MUTUAL FUND HISTORICAL DATA FROM YF AND ANALYZE USING FFF




LAB 203 START




In [17]:
# TO_PERIOD ('m') IS THEN CONVERTED TO TIMESTAMP OBJECT WHILE PLOTTING
ind_ew = erk.get_df('data/ind30_m_ew_rets.csv', to_per=True)
ind_vw = erk.get_df('data/ind30_m_vw_rets.csv', to_per=True)
sharpe_ratio_df = pd.DataFrame({'VW': erk.risk_info(ind_vw['1945':], only_sharpe=True),
                               'EW': erk.risk_info(ind_ew['1945':], only_sharpe=True)})


PLOT VW AND EW SHARPE RATIO




In [18]:
sr_data = [go.Bar(x=sharpe_ratio_df.index,
                  y=sharpe_ratio_df[col]) for col in sharpe_ratio_df.columns]
pyo.iplot(sr_data)

% OF CASES WHERE SR(EW) > SR(VW)




In [19]:
(sharpe_ratio_df['EW'] > sharpe_ratio_df['VW']).sum()/sharpe_ratio_df.shape[0]

0.6333333333333333

ROLLING WINDOW SR CALC FOR VW AND EW MARKET PF




In [20]:
sharpe_ratio_rolling_df = pd.DataFrame({
    'VW': ind_vw.rolling(window='1825D').apply(erk.risk_info, kwargs={'only_sharpe': True, 'rf': 0.03}, raw=True).mean(axis=1)['1945':].dropna(),
    'EW': ind_ew.rolling(window='1825D').apply(erk.risk_info, kwargs={'only_sharpe': True, 'rf': 0.03}, raw=True).mean(axis=1)['1945':].dropna()
})


PLOT ROLLING SR




In [1]:
sr_r_data = [go.Scatter(x=sharpe_ratio_rolling_df.index.to_timestamp(),
                      y=sharpe_ratio_rolling_df[col],
                      name=col) for col in sharpe_ratio_df.columns]
pyo.iplot(sr_r_data)

NameError: name 'sharpe_ratio_df' is not defined

LAB START 204




In [5]:
ind_vw_74 = erk.get_df('data/ind49_m_vw_rets.csv', to_per=True, start_period='1974')
ind_mkt_wts_74 = erk.plot_corr_mktret(ind_ret_filename='data/ind49_m_vw_rets.csv',
                                       n_firms_filename='data/ind49_m_nfirms.csv',
                                       size_filename='data/ind49_m_size.csv',
                                       start_period='1974',
                                       end_period=None,
                                       to_per=True,
                                       retrieve_mkt_cap_wts=True,
                                       format='%Y%m')